# Appendix: Data Cleaning Description

This project required a bit of data cleaning to get the final datasets that we worked with for our analyses. Here, we outline the steps undertaken to get the workable datasets that we used.

## Members of Congress Dataset

We will use [the @unitedstates project](https://github.com/unitedstates/congress-legislators) to get broad data on every single member of congress in our dataset. They provide downloadable CSVs which we can use to cross-reference names from the SEC Filing data to get different features with which we can run our hypothesis tests. We will need to prune down this dataset intensely in order to only contain the legislators that have filed with the SEC, instead of all legislators since the 1700s.

First we will make the requisite imports.

In [ ]:
import pandas as pd

We will load in the dataset. Because the data is seperated for current and historical legislators, we will need to load both of them in separately and then concatenate them together. We will also use this moment to remove some extraneous columns.

In [29]:
legislators_current = pd.read_csv("data/raw/legislators-current.csv")
legislators_historical = pd.read_csv("data/raw/legislators-historical.csv")

In [31]:
legislators = legislators_historical.append(legislators_current)
legislators = legislators[["last_name", "first_name", "middle_name", "gender", "type", "party", "state", "district", "senate_class"]]
legislators.head()

,last_name,first_name,middle_name,gender,type,party,state,district,senate_class
0,Bassett,Richard,NaN,M,sen,Anti-Administration,DE,NaN,2.0
1,Bland,Theodorick,NaN,M,rep,NaN,VA,9.0,NaN
2,Burke,Aedanus,NaN,M,rep,NaN,SC,2.0,NaN
3,Carroll,Daniel,NaN,M,rep,NaN,MD,6.0,NaN
4,Clymer,George,NaN,M,rep,NaN,PA,-1.0,NaN


In order to make the data easier to work with, we will separate the legislators dataset into Senators and Representatives. Note that we already need to deal with cherry-picking names, as some members have served terms in both the house and the senate, but the dataset only counts the most recent term served.

In [32]:
senators = legislators[legislators['type'] == "sen"]
representatives = legislators[(legislators['type'] == "rep") | ((legislators["first_name"] == "Roger") & (legislators["last_name"] == "Marshall"))]

Now, in order to make searching the dataset easier, we join the first name, last name, and middle initial of the legislators to put the name in the same format as the SEC filings.

In [36]:
senators["name"] = pd.DataFrame.copy(senators.apply(lambda r: "{}{}{}".format(r["first_name"], " " if pd.isna(r["middle_name"]) else " {} ".format(r["middle_name"][0]), r["last_name"]), axis=1))
representatives["name"] = pd.DataFrame.copy(representatives.apply( lambda r: "{}{}{}".format(r["first_name"], " " if pd.isna(r["middle_name"]) else " {} ".format(r["middle_name"]), r["last_name"]), axis=1))

/Users/prathamrawat/git/2950-Final/venv/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.
/Users/prathamrawat/git/2950-Final/venv/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [38]:
senators.head()

,last_name,first_name,middle_name,gender,type,party,state,district,senate_class,name
0,Bassett,Richard,NaN,M,sen,Anti-Administration,DE,NaN,2.0,Richard Bassett
6,Dalton,Tristram,NaN,M,sen,Pro-Administration,MA,NaN,1.0,Tristram Dalton
7,Elmer,Jonathan,NaN,M,sen,Pro-Administration,NJ,NaN,1.0,Jonathan Elmer
8,Few,William,NaN,M,sen,Anti-Administration,GA,NaN,2.0,William Few
11,Grayson,William,NaN,M,sen,Anti-Administration,VA,NaN,1.0,William Grayson


In [37]:
representatives.head()

,last_name,first_name,middle_name,gender,type,party,state,district,senate_class,name
1,Bland,Theodorick,NaN,M,rep,NaN,VA,9.0,NaN,Theodorick Bland
2,Burke,Aedanus,NaN,M,rep,NaN,SC,2.0,NaN,Aedanus Burke
3,Carroll,Daniel,NaN,M,rep,NaN,MD,6.0,NaN,Daniel Carroll
4,Clymer,George,NaN,M,rep,NaN,PA,-1.0,NaN,George Clymer
5,Contee,Benjamin,NaN,M,rep,NaN,MD,3.0,NaN,Benjamin Contee


However, there are still discrepancies in the data. Therefore, we also need to edit the data by hand to ensure that the names are able to be cross referenced by the SEC filings.

In [63]:
senators.loc[senators["name"] == "Angus S King", "name"] = "Angus S King, Jr."
senators.loc[senators["name"] == "Tommy H Tuberville", "name"] = "Thomas H Tuberville"
senators.loc[senators["name"] == "Bill F Hagerty", "name"] = "William F Hagerty, Iv"
senators.loc[senators["name"] == "Mitch McConnell", "name"] = "A. Mitchell Mcconnell, Jr."
senators.loc[senators["name"] == "Jerry Moran", "name"] = "Jerry Moran,"
senators.loc[senators["name"] == "Ron Wyden", "name"] = "Ron L Wyden"
senators.loc[senators["name"] == "Dan Sullivan", "name"] = "Daniel S Sullivan"
senators.loc[senators["name"] == "Jacky Rosen", "name"] = "Jacklyn S Rosen"
senators.loc[senators["name"] == "Bill Cassidy", "name"] = "William Cassidy"
senators.loc[senators["name"] == "Timothy Kaine", "name"] = "Timothy M Kaine"
senators.loc[senators["name"] == "David Perdue", "name"] = "David A Perdue , Jr"
senators.loc[senators["name"] == "Tina F Smith", "name"] = "Tina Smith"
senators.loc[senators["name"] == "Ted Cruz", "name"] = "Rafael E Cruz"
senators.loc[senators["name"] == "Tom S Udall", "name"] = "Thomas Udall"
senators.loc[senators["name"] == "Thom Tillis", "name"] = "Thomas R Tillis"
senators.loc[senators["name"] == "Robert P Casey", "name"] = "Robert P Casey, Jr."
senators.loc[senators["name"] == "Joe Manchin", "name"] = "Joseph Manchin, Iii"
senators.loc[senators["name"] == "Roger Marshall", "name"] = "Roger W Marshall"
senators.loc[senators["name"] == "John F Reed", "name"] = "Jack Reed"
senators.loc[senators["name"] == "Roy Blunt", "name"] = "Roy D Blunt"

In [41]:
representatives.loc[representatives["name"] == "Abigail Davis Spanberger", "name"] = "Abigail Spanberger"
representatives.loc[representatives["name"] == "Andrew R. Garbarino", "name"] = "Andrew Garbarino"
representatives.loc[representatives["name"] == "Anthony Gonzalez", "name"] = "Anthony E. Gonzalez"
representatives.loc[representatives["name"] == "Ashley Hinson", "name"] = "Ashley Hinson Arenholz"
representatives.loc[representatives["name"] == "Barry Moore", "name"] = "Felix Barry Moore"
representatives.loc[representatives["name"] == "Bill J. Pascrell", "name"] = "Bill Pascrell"
representatives.loc[representatives["name"] == "Blake David Moore", "name"] = "Blake Moore"
representatives.loc[representatives["name"] == "Bradley Scott Schneider", "name"] = "Bradley S. Schneider"
representatives.loc[representatives["name"] == "Brian J. Mast", "name"] = "Brian Mast"
representatives.loc[representatives["name"] == "Brian M. Higgins", "name"] = "Brian Higgins"
representatives.loc[representatives["name"] == "C. Scott Franklin", "name"] = "Scott Franklin"
representatives.loc[representatives["name"] == "Carol D. Miller", "name"] = "Carol Devine Miller"
representatives.loc[representatives["name"] == "Charles J. Fleischmann", "name"] = 'Charles J. "Chuck" Fleischmann'
representatives.loc[representatives["name"] == "Chris Jacobs", "name"] = "Christopher L. Jacobs"
representatives.loc[representatives["name"] == "Dan Crenshaw", "name"] = "Daniel Crenshaw"
representatives.loc[representatives["name"] == "David J. Trone", "name"] = "David Trone"
representatives.loc[representatives["name"] == "David Rouzer", "name"] = "David Cheston Rouzer"
representatives.loc[representatives["name"] == "Deborah Koff Ross", "name"] = "Deborah K. Ross"
representatives.loc[representatives["name"] == "Donna E. Shalala", "name"] = "Donna Shalala"
representatives.loc[representatives["name"] == "Earl L. Carter", "name"] = "Earl Leroy Carter"
representatives.loc[representatives["name"] == "Elaine G. Luria", "name"] = "Elaine Luria"
representatives.loc[representatives["name"] == "Frank J. Pallone", "name"] = "Frank Pallone"
representatives.loc[representatives["name"] == "Fred Stephen Upton", "name"] = "Fred Upton"
representatives.loc[representatives["name"] == "Gilbert Ray Cisneros", "name"] = "Gilbert Cisneros"
representatives.loc[representatives["name"] == "Harley Rouda", "name"] = "Harley E. Rouda"
representatives.loc[representatives["name"] == "Harold Rogers", "name"] = "Harold Dallas Rogers"
representatives.loc[representatives["name"] == "J. French Hill", "name"] = "James French Hill"
representatives.loc[representatives["name"] == "Jim Hagedorn", "name"] = "James Hagedorn"
representatives.loc[representatives["name"] == "John H. Rutherford", "name"] = "John Rutherford"
representatives.loc[representatives["name"] == "John R. Curtis", "name"] = "John Curtis"
representatives.loc[representatives["name"] == "Judy M. Chu", "name"] = "Judy Chu"
representatives.loc[representatives["name"] == "Kathy Ellen Manning", "name"] = "Kathy Manning"
representatives.loc[representatives["name"] == "Kenny Ewell Marchant", "name"] = "Kenny Marchant"
representatives.loc[representatives["name"] == "Kevin R. Hern", "name"] = "Kevin Hern"
representatives.loc[representatives["name"] == "Linda T. Sánchez", "name"] = "Linda T. Sanchez"
representatives.loc[representatives["name"] == "Lloyd A. Doggett", "name"] = "Lloyd Doggett"
representatives.loc[representatives["name"] == "Lloyd Smucker", "name"] = "Lloyd K. Smucker"
representatives.loc[representatives["name"] == "Michael Guest", "name"] = "Michael Patrick Guest"
representatives.loc[representatives["name"] == "Mike Gallagher", "name"] = "Michael John Gallagher"
representatives.loc[representatives["name"] == "Mike Garcia", "name"] = "Michael Garcia"
representatives.loc[representatives["name"] == "Patrick Edward Fallon", "name"] = "Patrick Fallon"
representatives.loc[representatives["name"] == "Pete A. Sessions", "name"] = "Pete Sessions"
representatives.loc[representatives["name"] == "Peter James Meijer", "name"] = "Peter Meijer"
representatives.loc[representatives["name"] == "Raja Krishnamoorthi", "name"] = "S. Raja Krishnamoorthi"
representatives.loc[representatives["name"] == "Rick W. Allen", "name"] = "Richard W. Allen"
representatives.loc[representatives["name"] == "Ro Khanna", "name"] = "Rohit Khanna"
representatives.loc[representatives["name"] == "Robert C. Scott", "name"] = 'Robert C. "Bobby" Scott'
representatives.loc[representatives["name"] == "Roger Marshall", "name"] = "Roger W. Marshall"
representatives.loc[representatives["name"] == "Stephanie I. Bice", "name"] = "Stephanie Bice"
representatives.loc[representatives["name"] == "Steve J. Chabot", "name"] = "Steve Chabot"
representatives.loc[representatives["name"] == "TJ Cox", "name"] = "TJ John (Tj) Cox"
representatives.loc[representatives["name"] == "Thomas R. Suozzi", "name"] = "Thomas Suozzi"
representatives.loc[representatives["name"] == "Tom O’Halleran", "name"] = "Tom O'Halleran"
representatives.loc[representatives["name"] == "Van Taylor", "name"] = "Nicholas Van Taylor"
representatives.loc[representatives["name"] == "W. Gregory Steube", "name"] = "Greg Steube"
representatives.loc[representatives["name"] == "Mark E. Green", "name"] = "Mark Green"
representatives.loc[representatives["name"] == "Jim Banks", "name"] = "James E. Banks"
representatives.loc[representatives["name"] == "Donald S. Beyer", "name"] = "Donald Sternoff Beyer"

/Users/prathamrawat/git/2950-Final/venv/lib/python3.7/site-packages/pandas/core/indexing.py:1817: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(loc, value, pi)


However, in order to complete the process, we will also pull in the SEC Filing data to reference names further, particularly those that are inconsistently referenced..

In [74]:
senate = pd.read_csv("data/raw/all_transactions.csv")
senate = senate.loc[senate.asset_type == "Stock"].loc[senate.ticker != "--"].dropna()
senate = senate.drop(columns=["ptr_link", "asset_type", "disclosure_date", "asset_description"])

house = pd.read_csv("data/raw/all_transactions_house.csv")
house = house.drop([1956, 3381, 8975, 8976])
house = house.loc[house.ticker != "--"].dropna()
house = house.drop(columns=["ptr_link", "disclosure_date", "disclosure_year", "asset_description"])
house['transaction_date'] = pd.to_datetime(house['transaction_date'])
house['transaction_year'] = pd.DatetimeIndex(house['transaction_date']).year
house['transaction_month'] = pd.DatetimeIndex(house['transaction_date']).month
house['transaction_day'] = pd.DatetimeIndex(house['transaction_date']).day
house["amount_lower"] = pd.to_numeric( house["amount"].replace("Over ", "", regex=True).replace("\$", "", regex=True).replace(" -.*$", "", regex=True).replace(",", "", regex=True))
house["amount_upper"] = pd.to_numeric( house["amount"].replace("Over ", "", regex=True).replace("-$", "- 0", regex=True).replace(".* - ", "", regex=True).replace("\$", "", regex=True).replace( ",", "", regex=True))
house["representative"] = house["representative"].str.split(" ", 1).str[1]

Now we will reference senator and representative names to fix some remaining issues with the data.

In [75]:
def get_senator(name):
    if name == "Ladda Tammy Duckworth":
        name = "Tammy Duckworth"
    if name == "Michael  B Enzi":
        name = "Michael B Enzi"
    if name == "Ron Wyden":
        name = "Ron L Wyden"
    if name == "John F Reed":
        name = "Jack Reed"
    if name == "Angus S King":
        name = "Angus S King, Jr."
    if name == "Roy Blunt":
        name = "Roy D Blunt"
    return indexed_senators.loc[name]

indexed_senators = senators.set_index("name")
indexed_senators["name"] = indexed_senators.index
trading_senators = senate["senator"].apply(get_senator)
trading_senators = trading_senators.drop_duplicates()
trading_senators = trading_senators.reset_index()
trading_senators = trading_senators.drop("index", axis=1)

In [76]:
representatives.loc[representatives["name"] == "Abigail Davis Spanberger", "name"] = "Abigail Spanberger"
representatives.loc[representatives["name"] == "Andrew R. Garbarino", "name"] = "Andrew Garbarino"
representatives.loc[representatives["name"] == "Anthony Gonzalez", "name"] = "Anthony E. Gonzalez"
representatives.loc[representatives["name"] == "Ashley Hinson", "name"] = "Ashley Hinson Arenholz"
representatives.loc[representatives["name"] == "Barry Moore", "name"] = "Felix Barry Moore"
representatives.loc[representatives["name"] == "Bill J. Pascrell", "name"] = "Bill Pascrell"
representatives.loc[representatives["name"] == "Blake David Moore", "name"] = "Blake Moore"
representatives.loc[representatives["name"] == "Bradley Scott Schneider", "name"] = "Bradley S. Schneider"
representatives.loc[representatives["name"] == "Brian J. Mast", "name"] = "Brian Mast"
representatives.loc[representatives["name"] == "Brian M. Higgins", "name"] = "Brian Higgins"
representatives.loc[representatives["name"] == "C. Scott Franklin", "name"] = "Scott Franklin"
representatives.loc[representatives["name"] == "Carol D. Miller", "name"] = "Carol Devine Miller"
representatives.loc[representatives["name"] == "Charles J. Fleischmann", "name"] = 'Charles J. "Chuck" Fleischmann'
representatives.loc[representatives["name"] == "Chris Jacobs", "name"] = "Christopher L. Jacobs"
representatives.loc[representatives["name"] == "Dan Crenshaw", "name"] = "Daniel Crenshaw"
representatives.loc[representatives["name"] == "David J. Trone", "name"] = "David Trone"
representatives.loc[representatives["name"] == "David Rouzer", "name"] = "David Cheston Rouzer"
representatives.loc[representatives["name"] == "Deborah Koff Ross", "name"] = "Deborah K. Ross"
representatives.loc[representatives["name"] == "Donna E. Shalala", "name"] = "Donna Shalala"
representatives.loc[representatives["name"] == "Earl L. Carter", "name"] = "Earl Leroy Carter"
representatives.loc[representatives["name"] == "Elaine G. Luria", "name"] = "Elaine Luria"
representatives.loc[representatives["name"] == "Frank J. Pallone", "name"] = "Frank Pallone"
representatives.loc[representatives["name"] == "Fred Stephen Upton", "name"] = "Fred Upton"
representatives.loc[representatives["name"] == "Gilbert Ray Cisneros", "name"] = "Gilbert Cisneros"
representatives.loc[representatives["name"] == "Harley Rouda", "name"] = "Harley E. Rouda"
representatives.loc[representatives["name"] == "Harold Rogers", "name"] = "Harold Dallas Rogers"
representatives.loc[representatives["name"] == "J. French Hill", "name"] = "James French Hill"
representatives.loc[representatives["name"] == "Jim Hagedorn", "name"] = "James Hagedorn"
representatives.loc[representatives["name"] == "John H. Rutherford", "name"] = "John Rutherford"
representatives.loc[representatives["name"] == "John R. Curtis", "name"] = "John Curtis"
representatives.loc[representatives["name"] == "Judy M. Chu", "name"] = "Judy Chu"
representatives.loc[representatives["name"] == "Kathy Ellen Manning", "name"] = "Kathy Manning"
representatives.loc[representatives["name"] == "Kenny Ewell Marchant", "name"] = "Kenny Marchant"
representatives.loc[representatives["name"] == "Kevin R. Hern", "name"] = "Kevin Hern"
representatives.loc[representatives["name"] == "Linda T. Sánchez", "name"] = "Linda T. Sanchez"
representatives.loc[representatives["name"] == "Lloyd A. Doggett", "name"] = "Lloyd Doggett"
representatives.loc[representatives["name"] == "Lloyd Smucker", "name"] = "Lloyd K. Smucker"
representatives.loc[representatives["name"] == "Michael Guest", "name"] = "Michael Patrick Guest"
representatives.loc[representatives["name"] == "Mike Gallagher", "name"] = "Michael John Gallagher"
representatives.loc[representatives["name"] == "Mike Garcia", "name"] = "Michael Garcia"
representatives.loc[representatives["name"] == "Patrick Edward Fallon", "name"] = "Patrick Fallon"
representatives.loc[representatives["name"] == "Pete A. Sessions", "name"] = "Pete Sessions"
representatives.loc[representatives["name"] == "Peter James Meijer", "name"] = "Peter Meijer"
representatives.loc[representatives["name"] == "Raja Krishnamoorthi", "name"] = "S. Raja Krishnamoorthi"
representatives.loc[representatives["name"] == "Rick W. Allen", "name"] = "Richard W. Allen"
representatives.loc[representatives["name"] == "Ro Khanna", "name"] = "Rohit Khanna"
representatives.loc[representatives["name"] == "Robert C. Scott", "name"] = 'Robert C. "Bobby" Scott'
representatives.loc[representatives["name"] == "Roger Marshall", "name"] = "Roger W. Marshall"
representatives.loc[representatives["name"] == "Stephanie I. Bice", "name"] = "Stephanie Bice"
representatives.loc[representatives["name"] == "Steve J. Chabot", "name"] = "Steve Chabot"
representatives.loc[representatives["name"] == "TJ Cox", "name"] = "TJ John (Tj) Cox"
representatives.loc[representatives["name"] == "Thomas R. Suozzi", "name"] = "Thomas Suozzi"
representatives.loc[representatives["name"] == "Tom O’Halleran", "name"] = "Tom O'Halleran"
representatives.loc[representatives["name"] == "Van Taylor", "name"] = "Nicholas Van Taylor"

representatives.loc[representatives["name"] == "W. Gregory Steube", "name"] = "Greg Steube"
representatives.loc[representatives["name"] == "Mark E. Green", "name"] = "Mark Green"
representatives.loc[representatives["name"] == "Jim Banks", "name"] = "James E. Banks"
representatives.loc[representatives["name"] == "Donald S. Beyer", "name"] = "Donald Sternoff Beyer"

house.loc[house["representative"] == "W. Greg Steube"] = "Greg Steube"
house.loc[house["representative"] == "Neal Patrick Dunn MD, FACS"] = "Neal P. Dunn"
house.loc[house["representative"] == "Neal Patrick MD, FACS Dunn"] = "Neal P. Dunn"
house.loc[house["representative"] == "Neal Patrick MD, Facs Dunn"] = "Neal P. Dunn"
house.loc[house["representative"] == "Mark Dr Green"] = "Mark Green"
house.loc[house["representative"] == "James E Hon Banks"] = "James E. Banks"
house.loc[house["representative"] == "Donald Sternoff Honorable Beyer"] = "Donald Sternoff Beyer"

In [78]:
def get_rep(name):
    return indexed_representatives.loc[name]

indexed_representatives = representatives.set_index("name")
indexed_representatives["name"] = indexed_representatives.index
trading_representatives = house["representative"].apply(get_rep)
trading_representatives = trading_representatives.drop_duplicates()
trading_representatives = trading_representatives.reset_index()
trading_representatives = trading_representatives.drop("index", axis=1)

Finally, we can save this data as a CSV file.

In [80]:
trading_senators.to_csv("data/cleaned/senators.csv")
trading_representatives.to_csv("data/cleaned/representatives.csv")

## Stock Data

The `yfinance` API is a Python package that can be used to quickly download data for stock tickers into a `pandas` dataframe. Additional documentation on the API can be found [here](https://pypi.org/project/yfinance/). It provides daily stock prices, with the market opening, close, highs, lows, and volume for the day. We can prune through most of this data though and focus retrieving the datapoints of focus for this.

First, we make necessary imports.

In [4]:
import yfinance as yf
import pandas as pd

We can load in the raw Senate and House SEC filings. We will not need the cleaned versions to download the stock data.

In [2]:
senator_trades = pd.read_csv("data/cleaned/all_transactions_senate.csv")
senator_trades.head()

,Unnamed: 0,transaction_date,owner,ticker,type,amount,comment,senator,transaction_year,transaction_month,transaction_day,amount_lower,amount_upper
0,5,2021-08-18,Joint,SSYS,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,2021,8,18,1001,15000
1,6,2021-08-12,Joint,SSBK,Purchase,"$50,001 - $100,000",--,Thomas H Tuberville,2021,8,12,50001,100000
2,19,2021-08-23,Spouse,LMRK,Sale (Full),"$1,001 - $15,000",--,Thomas R Carper,2021,8,23,1001,15000
3,22,2021-08-11,Spouse,WW,Purchase,"$1,001 - $15,000",--,Thomas R Carper,2021,8,11,1001,15000
4,23,2021-08-04,Spouse,ARE,Sale (Partial),"$1,001 - $15,000",--,Thomas R Carper,2021,8,4,1001,15000


In [3]:
house_trades = pd.read_csv("data/cleaned/all_transactions_house.csv")
house_trades.head()

,Unnamed: 0,transaction_date,owner,ticker,type,amount,representative,district,cap_gains_over_200_usd,transaction_year,transaction_month,transaction_day,amount_lower,amount_upper
0,0,2021-09-27,joint,BP,purchase,"$1,001 - $15,000",Virginia Foxx,NC05,False,2021,9,27,1001,15000
1,1,2021-09-13,joint,XOM,purchase,"$1,001 - $15,000",Virginia Foxx,NC05,False,2021,9,13,1001,15000
2,2,2021-09-10,joint,ILPT,purchase,"$15,001 - $50,000",Virginia Foxx,NC05,False,2021,9,10,15001,50000
3,3,2021-09-28,joint,PM,purchase,"$15,001 - $50,000",Virginia Foxx,NC05,False,2021,9,28,15001,50000
4,4,2021-09-17,self,BLK,sale_partial,"$1,001 - $15,000",Alan S. Lowenthal,CA47,False,2021,9,17,1001,15000


Now, we will use `yfinance` to download the daily price data for all the stock tickers mentioned in the SEC filings.

In [6]:
senate_stocks = yf.download(" ".join(senator_trades["ticker"].unique()),
                            start=senator_trades["transaction_date"].min(),
                            end=senator_trades["transaction_date"].max())

[*********************100%***********************]  909 of 909 completed

130 Failed downloads:
- LINE: None
- CMCSK: None
- AVP: No data found, symbol may be delisted
- ICON: No data found, symbol may be delisted
- DPM: No data found for this date range, symbol may be delisted
- MDCA: No data found, symbol may be delisted
- UTX: No data found, symbol may be delisted
- ACE: None
- XLS-WI: None
- FNGN: No data found, symbol may be delisted
- IPCM: None
- AKRX: None
- CVC: None
- EMC: None
- SBUX.SW: No data found, symbol may be delisted
- TNGO: None
- Q: None
- CY: No data found, symbol may be delisted
- VIAB: No data found, symbol may be delisted
- APC: No data found, symbol may be delisted
- TMK: No data found, symbol may be delisted
- LLTC: None
- CTRL: No data found, symbol may be delisted
- KRFT: None
- BCR: None
- LB: No data found, symbol may be delisted
- CRZO: No data found, symbol may be delisted
- RDSA: No data found, symbol may be delisted
- BPL: No data found, symbol may be

In [19]:
house_stocks = yf.download(" ".join(house_trades["ticker"].unique()),
                            start=house_trades["transaction_date"].min(),
                            end=house_trades["transaction_date"].max())

[*********************100%***********************]  1560 of 1560 completed

108 Failed downloads:
- HS: None
- APPL: None
- BOA: None
- FSKR: No data found, symbol may be delisted
- BXS$A: No data found, symbol may be delisted
- PFPT: No data found, symbol may be delisted
- DCMYY: No data found, symbol may be delisted
- FLIR: No data found, symbol may be delisted
- EBJ: None
- SPKE: No data found, symbol may be delisted
- BROADCOM: No data found, symbol may be delisted
- AMTD: No data found, symbol may be delisted
- TFDXX: None
- WPX: No data found, symbol may be delisted
- BAC$K: No data found, symbol may be delisted
- CBS: No data found, symbol may be delisted
- DNKN: No data found, symbol may be delisted
- LBDAV: No data found, symbol may be delisted
- WAIR: No data found, symbol may be delisted
- RP: No data found, symbol may be delisted
- MTSC: No data found, symbol may be delisted
- PRSP: No data found, symbol may be delisted
- BRK.B: No data found, symbol may be delisted
- SNOXX

In [7]:
senate_stocks.head()

Adj Close                                                          \
             0QZI.IL 3V64.TI          A         AA      AAGIY        AAL AAN   
Date                                                                           
2012-09-13       NaN     NaN  25.213678  21.965357  13.293386  10.605844 NaN   
2012-09-14       NaN     NaN  26.100559  22.444349  13.489270   9.927068 NaN   
2012-09-17       NaN     NaN  26.034864  21.851311  13.311191   9.955351 NaN   
2012-09-18       NaN     NaN  25.982307  21.646027  13.355713   9.719666 NaN   
2012-09-19       NaN     NaN  26.199104  21.600407  13.222154  10.341875 NaN   

                                            ... Volume                  \
                AAON       AAPL        AAT  ...   YUMC ZAYO        ZBH   
Date                                        ...                          
2012-09-13  7.993820  21.064934  21.613337  ...    NaN  NaN  1377800.0   
2012-09-14  8.430821  21.320934  21.832767  ...    NaN  NaN  1864800.0   
2012-09-17  8.228810  21.583092  21.942476  ...    NaN  NaN   746200.0   
2012-09-18  8.418453  21.648787  21.848440  ...    NaN  NaN  1231800.0   
2012-09-19  8.348368  21.654642  21.550652  ...    NaN  NaN  1628200.0   

                                                            
                ZIOP  ZM        ZNGA ZNGA.SW ZTS ^MWE ^RGP  
Date                                                        
2012-09-13  518500.0 NaN  13112100.0     NaN NaN  NaN  NaN  
2012-09-14  502500.0 NaN  31370600.0     NaN NaN  NaN  NaN  
2012-09-17  568900.0 NaN  11066300.0     NaN NaN  NaN  NaN  
2012-09-18  855600.0 NaN   9374100.0     NaN NaN  NaN  NaN  
2012-09-19  569500.0 NaN  18435400.0     NaN NaN  NaN  NaN  

[5 rows x 5454 columns]

In [20]:
house_stocks.head()

Adj Close                                                 \
                        35G.SG         AA      AAGIY     AAIGF        AAL AAN   
Date                                                                            
2018-12-27 00:00:00  22.208635  27.100815  31.333757  6.931739  31.530161 NaN   
2018-12-28 00:00:00  22.305658  26.542034  31.728800  7.130778  31.323498 NaN   
2018-12-29 00:00:00        NaN        NaN        NaN       NaN        NaN NaN   
2018-12-30 00:00:00        NaN        NaN        NaN       NaN        NaN NaN   
2018-12-31 00:00:00        NaN  26.522078  31.680626  7.165393  31.599045 NaN   

                                                            ...     Volume  \
                          AAPL AAVMY         AB      ABALX  ...       YUMC   
Date                                                        ...              
2018-12-27 00:00:00  38.008892   NaN  20.222670  22.453785  ...   994100.0   
2018-12-28 00:00:00  38.028370   NaN  20.812433  22.481007  ...  1199900.0   
2018-12-29 00:00:00        NaN   NaN        NaN        NaN  ...        NaN   
2018-12-30 00:00:00        NaN   NaN        NaN        NaN  ...        NaN   
2018-12-31 00:00:00  38.395920   NaN  21.200434  22.589870  ...   795700.0   

                                                                               \
                             Z        ZBH      ZBRA        ZEN       ZION  ZM   
Date                                                                            
2018-12-27 00:00:00  2511000.0  1578900.0  504500.0  1119800.0  2534200.0 NaN   
2018-12-28 00:00:00  1541600.0  1860000.0  344800.0   953800.0  2558600.0 NaN   
2018-12-29 00:00:00        NaN        NaN       NaN        NaN        NaN NaN   
2018-12-30 00:00:00        NaN        NaN       NaN        NaN        NaN NaN   
2018-12-31 00:00:00  1882400.0  1389700.0  409100.0   921100.0  2575600.0 NaN   

                                                 
                           ZNGA ZOOM        ZTS  
Date                                             
2018-12-27 00:00:00  14644000.0  NaN  2244700.0  
2018-12-28 00:00:00   7666400.0  NaN  1797300.0  
2018-12-29 00:00:00         NaN  NaN        NaN  
2018-12-30 00:00:00         NaN  NaN        NaN  
2018-12-31 00:00:00  10814800.0  NaN  1485200.0  

[5 rows x 9360 columns]

We can see that a few stock tickers were not found. This can be due to many different things such as previously public companies going private, bankruptcy, mergers and acquisitions, and other market events. We can ignore these datapoints, we still have a lot of data to work with.

There are also a lot of `NaN` values in the dataset. This makese sense though, as not all companies might have been publically listed in the timeframe given. This is not a problem in our dataset though, members of congress cannot trade stocks that do not exist yet, so this should not affect our analysis at all.

We will remove irrelevant columns in the data to only keep what we need.

In [8]:
senate_stocks = senate_stocks.drop(["Open", "Adj Close", "Low", "High"], axis=1)
senate_stocks.head()

Close                                                            \
           0QZI.IL 3V64.TI          A         AA  AAGIY    AAL AAN      AAON   
Date                                                                           
2012-09-13     NaN     NaN  27.453505  23.140890  14.93  11.25 NaN  8.617778   
2012-09-14     NaN     NaN  28.419170  23.645519  15.15  10.53 NaN  9.088889   
2012-09-17     NaN     NaN  28.347639  23.020741  14.95  10.56 NaN  8.871111   
2012-09-18     NaN     NaN  28.290415  22.804470  15.00  10.31 NaN  9.075556   
2012-09-19     NaN     NaN  28.526466  22.756411  14.85  10.97 NaN  9.000000   

                                  ... Volume                                \
                 AAPL        AAT  ...   YUMC ZAYO        ZBH      ZIOP  ZM   
Date                              ...                                        
2012-09-13  24.392143  27.580000  ...    NaN  NaN  1377800.0  518500.0 NaN   
2012-09-14  24.688572  27.860001  ...    NaN  NaN  1864800.0  502500.0 NaN   
2012-09-17  24.992144  28.000000  ...    NaN  NaN   746200.0  568900.0 NaN   
2012-09-18  25.068214  27.879999  ...    NaN  NaN  1231800.0  855600.0 NaN   
2012-09-19  25.075001  27.500000  ...    NaN  NaN  1628200.0  569500.0 NaN   

                                              
                  ZNGA ZNGA.SW ZTS ^MWE ^RGP  
Date                                          
2012-09-13  13112100.0     NaN NaN  NaN  NaN  
2012-09-14  31370600.0     NaN NaN  NaN  NaN  
2012-09-17  11066300.0     NaN NaN  NaN  NaN  
2012-09-18   9374100.0     NaN NaN  NaN  NaN  
2012-09-19  18435400.0     NaN NaN  NaN  NaN  

[5 rows x 1818 columns]

In [21]:
house_stocks = house_stocks.drop(["Open", "Adj Close", "Low", "High"], axis=1)
house_stocks.head()

Close                                         \
                        35G.SG     AA      AAGIY AAIGF        AAL AAN   
Date                                                                    
2018-12-27 00:00:00  22.889999  27.16  32.520000  8.01  32.040001 NaN   
2018-12-28 00:00:00  22.990000  26.60  32.930000  8.24  31.830000 NaN   
2018-12-29 00:00:00        NaN    NaN        NaN   NaN        NaN NaN   
2018-12-30 00:00:00        NaN    NaN        NaN   NaN        NaN NaN   
2018-12-31 00:00:00        NaN  26.58  32.880001  8.28  32.110001 NaN   

                                                            ...     Volume  \
                          AAPL AAVMY         AB      ABALX  ...       YUMC   
Date                                                        ...              
2018-12-27 00:00:00  39.037498   NaN  26.059999  24.750000  ...   994100.0   
2018-12-28 00:00:00  39.057499   NaN  26.820000  24.780001  ...  1199900.0   
2018-12-29 00:00:00        NaN   NaN        NaN        NaN  ...        NaN   
2018-12-30 00:00:00        NaN   NaN        NaN        NaN  ...        NaN   
2018-12-31 00:00:00  39.435001   NaN  27.320000  24.900000  ...   795700.0   

                                                                               \
                             Z        ZBH      ZBRA        ZEN       ZION  ZM   
Date                                                                            
2018-12-27 00:00:00  2511000.0  1578900.0  504500.0  1119800.0  2534200.0 NaN   
2018-12-28 00:00:00  1541600.0  1860000.0  344800.0   953800.0  2558600.0 NaN   
2018-12-29 00:00:00        NaN        NaN       NaN        NaN        NaN NaN   
2018-12-30 00:00:00        NaN        NaN       NaN        NaN        NaN NaN   
2018-12-31 00:00:00  1882400.0  1389700.0  409100.0   921100.0  2575600.0 NaN   

                                                 
                           ZNGA ZOOM        ZTS  
Date                                             
2018-12-27 00:00:00  14644000.0  NaN  2244700.0  
2018-12-28 00:00:00   7666400.0  NaN  1797300.0  
2018-12-29 00:00:00         NaN  NaN        NaN  
2018-12-30 00:00:00         NaN  NaN        NaN  
2018-12-31 00:00:00  10814800.0  NaN  1485200.0  

[5 rows x 3120 columns]

Now that we have this data, we can save it to a CSV file for use in our analyses, and in other data-cleaning proceses.

In [23]:
senate_stocks.to_csv("data/cleaned/senate_stocks.csv")
house_stocks.to_csv("data/cleaned/house_stocks.csv")

We will also download data from the S&P 500, an index fund that tracks the prices of the largest companies in the New York Stock Exchange. It is a common benchmark for market performance. Using the ticker `SPY`, we can download the prices for the index fund over the range of our SEC filings.

In [25]:
spy = yf.download("spy",
            min(senator_trades["transaction_date"].min(), house_trades["transaction_date"].min()),
            max(senator_trades["transaction_date"].max(), house_trades["transaction_date"].max())
            )
spy.head()

[*********************100%***********************]  1 of 1 completed


,Open,High,Low,Close,Adj Close,Volume
Date,,,,,,
2012-09-13,144.369995,147.039993,143.990005,146.589996,123.021996,225470200
2012-09-14,146.880005,148.110001,146.759995,147.240005,123.567558,169777000
2012-09-17,146.940002,147.190002,146.369995,146.740005,123.147896,119427800
2012-09-18,146.490005,146.809998,146.250000,146.619995,123.047195,98326600
2012-09-19,146.789993,147.169998,146.410004,146.699997,123.114342,128318300


In [27]:
spy.drop(["Open", "Adj Close", "Low", "High"], axis=1)

,Close,Volume
Date,,
2012-09-13,146.589996,225470200
2012-09-14,147.240005,169777000
2012-09-17,146.740005,119427800
2012-09-18,146.619995,98326600
2012-09-19,146.699997,128318300
...,...,...
2021-09-23,443.179993,76396000
2021-09-24,443.910004,62094800
2021-09-27,442.640015,61371100


In [28]:
spy.to_csv("data/cleaned/spy.csv")


## Senate SEC Filings

The main dataset of SEC filings is pulled from [Senate Stock Watcher](https://senatestockwatcher.com/api) which provides an API for following Senator's stock trades. By law, officeholding members of the government have to disclose every stock trade that they make to the Securities and Exchange Commission. By following these trades, we hope to find some interesting results. The site offers a realtime API that updates every time an SEC filing is made. However, it also offers historical data in a downloadable CSV format, which is what we are using for our analysis.

We will first load the dataset into a dataframe. We will also be referencing data from the downloaded stock data.

In [97]:
senate = pd.read_csv("data/raw/all_transactions.csv")
senate.head()

,transaction_date,owner,ticker,asset_description,asset_type,type,amount,comment,senator,ptr_link,disclosure_date
0,01/16/2021,Joint,--,Fitso (Exchanged) <br> Zomato (Received) <div ...,Non-Public Stock,Exchange,"$1,001 - $15,000",Exchange of stock due to merger. Asset is unde...,Patrick J Toomey,https://efdsearch.senate.gov/search/view/ptr/4...,09/09/2021
1,08/30/2021,Joint,CALX,"Calix, Inc Common Stock <div class=""text-muted...",Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/d...,09/08/2021
2,08/30/2021,Joint,ECOM,ChannelAdvisor Corporation Common Stock <div c...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/d...,09/08/2021
3,08/30/2021,Joint,ECOM,ChannelAdvisor Corporation Common Stock <div c...,Stock Option,Sale (Full),"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/d...,09/08/2021
4,08/27/2021,Joint,CALX,"Calix, Inc Common Stock <div class=""text-muted...",Stock Option,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,https://efdsearch.senate.gov/search/view/ptr/d...,09/08/2021


We can drop rows with bad data, and columns with unnecessary data.

In [98]:
senate = senate.loc[senate.asset_type == "Stock"].loc[senate.ticker != "--"].dropna()
senate = senate.drop(columns=["ptr_link", "asset_type", "disclosure_date", "asset_description"])

We now focus on making sure the data is of the correct type. We will convert the dates into proper date objects, and convert the magnitude of the trades into numerical upper and lower bounds.

In [99]:
senate["transaction_date"] = pd.to_datetime(senate["transaction_date"])
senate['transaction_year'] = pd.DatetimeIndex(senate['transaction_date']).year
senate['transaction_month'] = pd.DatetimeIndex(senate['transaction_date']).month
senate['transaction_day'] = pd.DatetimeIndex(senate['transaction_date']).day
senate.sort_values("transaction_date", ascending=False).head()

,transaction_date,owner,ticker,type,amount,comment,senator,transaction_year,transaction_month,transaction_day
19,2021-08-23,Spouse,LMRK,Sale (Full),"$1,001 - $15,000",--,Thomas R Carper,2021,8,23
5,2021-08-18,Joint,SSYS,Purchase,"$1,001 - $15,000",--,Thomas H Tuberville,2021,8,18
6,2021-08-12,Joint,SSBK,Purchase,"$50,001 - $100,000",--,Thomas H Tuberville,2021,8,12
22,2021-08-11,Spouse,WW,Purchase,"$1,001 - $15,000",--,Thomas R Carper,2021,8,11
23,2021-08-04,Spouse,ARE,Sale (Partial),"$1,001 - $15,000",--,Thomas R Carper,2021,8,4


In [100]:
senate["amount_lower"] = pd.to_numeric(
    senate["amount"]
        .replace("Over ", "", regex=True)
        .replace("\$", "", regex=True)
        .replace(" -.*$", "", regex=True)
        .replace(",", "", regex=True)
)

senate["amount_upper"] = pd.to_numeric(
    senate["amount"]
        .replace("Over ", "", regex=True)
        .replace(".* - ", "", regex=True)
        .replace("\$", "", regex=True)
        .replace(",", "", regex=True)
)

Now that the data is clean, we can parse through it to isolate traded stocks, and attach a value to them.

The SEC filings only give us assorted buying and selling events of stock. Here, we parse through the buys and sells and try to match the buy events to sell events to try to assign dates and values to these investments, which we can then use to make much more insightful analyses with the data.

In [101]:
trades = []
for senator in senate["senator"].unique():
    data = senate[senate["senator"] == senator]
    data = data.sort_values("transaction_date", ascending=True)

    temp = {}

    for row in data.itertuples():
        weight = (row.amount_lower + row.amount_upper) / 2
        if row.type == "Sale (Full)":
            if row.ticker in temp and row.ticker in senate_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = senate_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = senate_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                    temp.pop(row.ticker)
                except KeyError as e:
                    print(e)
        elif row.type == "Sale (Partial)":
            if row.ticker in temp:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = senate_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = senate_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                except KeyError as e:
                    print(e)
        elif row.type == "Purchase":
            row_dict = {}
            for field in senate.columns:
                row_dict[field] = getattr(row, field)
            temp[row.ticker] = row_dict

senator_trades = pd.DataFrame(trades)

Timestamp('2020-02-22 00:00:00')
Timestamp('2020-02-22 00:00:00')
Timestamp('2019-01-19 00:00:00')


Now that we have the a percentage gain/loss attached to each trade, we can run proper analyses on the trades made. We can save this data to a CSV.

In [ ]:
senator_trades.to_csv("data/cleaned/senator_trades.csv")s

## House of Representatives SEC Filings

Similar to the data for the Senate, members of the House are also required to disclose their stock market trades while holding office. This data was pulled from a similar online service to the Senate data: [House Stock Watcher](https://housestockwatcher.com). They also provide an API for following trades, along with a downloadable CSV format for historical data. This CSV is what we used for our analysis.

We treat this data essentially the same as we did with the data for the Senate. First, load the raw dataset itself.

In [102]:
house = pd.read_csv("data/raw/all_transactions_house.csv")
house.head()

,disclosure_year,disclosure_date,transaction_date,owner,ticker,asset_description,type,amount,representative,district,ptr_link,cap_gains_over_200_usd
0,2021,10/04/2021,2021-09-27,joint,BP,BP plc,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
1,2021,10/04/2021,2021-09-13,joint,XOM,Exxon Mobil Corporation,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
2,2021,10/04/2021,2021-09-10,joint,ILPT,Industrial Logistics Properties Trust - Common...,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
3,2021,10/04/2021,2021-09-28,joint,PM,Phillip Morris International Inc,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,https://disclosures-clerk.house.gov/public_dis...,False
4,2021,10/04/2021,2021-09-17,self,BLK,BlackRock Inc,sale_partial,"$1,001 - $15,000",Hon. Alan S. Lowenthal,CA47,https://disclosures-clerk.house.gov/public_dis...,False


We remove rows with bad data and extraneous columns.

In [103]:
house = house.drop([1956, 3381, 8975, 8976])

house = house.loc[house.ticker != "--"].dropna()
house = house.drop(columns=["ptr_link", "disclosure_date", "disclosure_year", "asset_description"])
house.head()

,transaction_date,owner,ticker,type,amount,representative,district,cap_gains_over_200_usd
0,2021-09-27,joint,BP,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,False
1,2021-09-13,joint,XOM,purchase,"$1,001 - $15,000",Hon. Virginia Foxx,NC05,False
2,2021-09-10,joint,ILPT,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,False
3,2021-09-28,joint,PM,purchase,"$15,001 - $50,000",Hon. Virginia Foxx,NC05,False
4,2021-09-17,self,BLK,sale_partial,"$1,001 - $15,000",Hon. Alan S. Lowenthal,CA47,False


Now we can fix the datatypes for the dates, and the upper and lower bounds for the stock amount traded.

In [104]:
house['transaction_date'] = pd.to_datetime(house['transaction_date'])
house['transaction_year'] = pd.DatetimeIndex(house['transaction_date']).year
house['transaction_month'] = pd.DatetimeIndex(house['transaction_date']).month
house['transaction_day'] = pd.DatetimeIndex(house['transaction_date']).day
house["amount_lower"] = pd.to_numeric( house["amount"].replace("Over ", "", regex=True).replace("\$", "", regex=True).replace(" -.*$", "", regex=True).replace(",", "", regex=True))
house["amount_upper"] = pd.to_numeric( house["amount"].replace("Over ", "", regex=True).replace("-$", "- 0", regex=True).replace(".* - ", "", regex=True).replace("\$", "", regex=True).replace( ",", "", regex=True))
house["representative"] = house["representative"].str.split(" ", 1).str[1]

Now we can parse through the data to isolate buying and selling of stock, the same way we did with the Senate filings.

In [105]:
trades = []
for representative in house["representative"].unique():
    data = house[house["representative"] == representative]
    data = data.sort_values("transaction_date", ascending=True)
    temp = {}
    for row in data.itertuples():
        weight = (row.amount_lower + row.amount_upper) / 2
        if row.type == "sale_full":
            if row.ticker in temp and row.ticker in house_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = house_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = house_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                    temp.pop(row.ticker)
                except KeyError as e:
                    print(e)
        elif row.type == "sale_partial":
            if row.ticker in temp and row.ticker in house_stocks["Close"]:
                try:
                    temp[row.ticker]["sell_date"] = row.transaction_date
                    temp[row.ticker]["weight"] = weight
                    buy = house_stocks["Close"][row.ticker].loc[temp[row.ticker]["transaction_date"]]
                    sell = house_stocks["Close"][row.ticker].loc[row.transaction_date]
                    temp[row.ticker]["gain"] = (sell - buy) / buy
                    trades.append(temp[row.ticker])
                except KeyError as e:
                    print(e)
        elif row.type == "purchase":
            row_dict = {}
            for field in house.columns:
                row_dict[field] = getattr(row, field)
            temp[row.ticker] = row_dict

house_trades = pd.DataFrame(trades)

And finally, we can save the resulting dataframe to a CSV.

In [ ]:
house_trades.to_csv("data/cleaned/house_trades.csv")